### 1. Get patches & downsampled images

In [ ]:
ndpi_samples = {  
    # 'Patient ID': {
        # 'key': ndpi open key, 
        # 'nums': [list of histo slice IDs]}
}

In [ ]:
from create_patches import *

SCALE_IMG = True
PATCHES   = False   

SIZE  = 512
START = 0

folder_in      = ''  
folder_down    = ''
folder_patches = ''
            
if SCALE_IMG:
    for sid in ndpi_samples:
        print(sid, ':')
        for s in ndpi_samples[sid]['nums']:
            print('- ', s)
            scale_hsv(folder_in + sid + '/', folder_down, sid, s, target_means, key=ndpi_samples[sid]['key'])    
            
if PATCHES:
    for sid in ndpi_samples:
        print(sid, ':')
        for s in slices[sid]['nums']:
            print('- ', s)
            img_means = check_hsv(folder_in + sid + '/', sid, s, key=ndpi_samples[sid]['key'])
            divide_image(folder_in + sid + '/', folder_patches+sid+'_'+s+'_SN/', sid+'_'+s, target_means, img_means, size=SIZE, key=slices[sid]['key'], start=START)   


### 2. Count cell nuclei per patch in QuPath

Run the cell nuclei counting script on QuPath

### 3. Get tissue fractions in ImagePro

Run the tissue fraction segmentation script on ImagePro.
Note - We use large patches here (5120x5120 pixels) and then break them down into smaller patches in the next step. This proved to be more efficient.

### 4. Get cell density and tissue fraction maps

We assume previous analysis has resulted in:
- A csv file with the number of cell nuclei per patch
- A txt file with the tissue fractions per patch

In [ ]:
param_samples = {  
    # 'name of cell density csv file': [
        # name of tissue fraction txt file, 
        # height, 
        # width, 
        # flip vertically (opt), 
        # flip horizontally (opt)]
}

In [ ]:
from get_histo_param_maps import get_txt_from_masks, get_density_and_fractions, resize_maps

PLOT   = False
folder_original     = ''
folder_downsampled  = ''

folder_ndpi  = 'F:/Histology/NDPI/'  
folder_masks = 'F:/Histology/PNG/'  
      
padding = [0,4000,0,0]
fractions = ['stroma', 'lumen', 'epithelial']
  
for sid in ndpi_samples:
    for s in ndpi_samples[sid]['nums']:
        get_txt_from_masks(folder_ndpi, folder_masks, sid, s, padding, key=ndpi_samples[sid]['key'])
        
for sid in param_samples:
    for s in param_samples[sid]['nums']:
        get_density_and_fractions(folder_original, param_samples[sid], sid, plot=PLOT)         

for sid in ndpi_samples:
    for s in ndpi_samples[sid]['nums']:
        for fraction in fractions:
            resize_maps(folder_original, folder_downsampled, sid, s, fraction)

### 5. Compare ROIs in histology and VERDICT maps 

In [ ]:
MRI_samples = { 
    # 'Patient ID': 'slice ID',
}

multiple_lesions = {
    # 'Patient ID': 'L2'
}

GG = {
    # 'GG 3+4':   [ list of patient IDs ],
    # 'GG 3+4+5': [ list of patient IDs ],
    # 'GG 4+3':   [ list of patient IDs ],
    # 'GG >4+3':  [ list of patient IDs ]
}

In [ ]:
from get_ROI_stats import *

##### Define parameters
results  = {}
MR_map   = 'fIC'
seq      = 'VERDICT' # VERDICT or mpMRI
PLOT     = False
VERBOSE  = False
folder_MRI   = ''  
folder_histo = ''

#### Get sample sids
df = get_all_roi_stats(MR_map, seq)
slices = [i for i in MRI_samples.keys()]
all_slices = slices.copy()

#### Process data
for sid in slices:
   if VERBOSE:
      print('\n','-'*5,sid,'-'*5)
   
   # For cases with multiple lesions, choose one
   try:
      cancer = multiple_lesions[sid]
   except:
      cancer = 'L1'
      
   try:
      this_res = {}
      this_res['fIC-cancer'],        this_res['fIC-benign']        = get_sid_roi(df, sid, MR_map, cancer_label=cancer, verbose=VERBOSE)
      this_res['density-cancer'],    this_res['density-benign']    = analyse_density(folder_histo, sid, MRI_samples[sid],                        plot=PLOT, cancer=cancer, verbose=VERBOSE)
      this_res['epithelial-cancer'], this_res['epithelial-benign'] = analyse_density(folder_histo, sid, MRI_samples[sid], hist_name='epithelial',plot=PLOT, cancer=cancer, verbose=VERBOSE)
      this_res['stroma-cancer'],     this_res['stroma-benign']     = analyse_density(folder_histo, sid, MRI_samples[sid], hist_name='stroma',    plot=PLOT, cancer=cancer, verbose=VERBOSE)
      this_res['lumen-cancer'],      this_res['lumen-benign']      = analyse_density(folder_histo, sid, MRI_samples[sid], hist_name='lumen',     plot=PLOT, cancer=cancer, verbose=VERBOSE)
      results[sid] = this_res
   except:
      print('Error in ', sid)
      all_slices.remove(sid)

fIC_vals, density_vals, epith_vals, stroma_vals, lumen_vals = results_to_numpy_arrays(results)

epith_density_vals = get_fraction_density(density_vals,epith_vals,stroma_vals)

Analyse the difference between benign & csPCa ROIs for fIC/ADC and histologically derived parameters. 

In [ ]:
vals  = (fIC_vals, density_vals, epith_vals, stroma_vals, lumen_vals)
names = [MR_map, 'Cell density', 'Epithelial fraction', 'Stromal fraction', 'Luminal fraction']

_, ax = plt.subplots(1,5, figsize=(15,4))
for i, data, name in zip(range(len(vals)), vals, names):
    boxplot_single(ax[i], data, name)
    get_auc(data, name)

Plot the correlation between fIC/ADC and histologically derived parameters. 

In [ ]:
vals  = (density_vals, epith_density_vals, epith_vals, stroma_vals, lumen_vals)
names = ['Cell density','Epithelial cell density', 'Epithelial fraction', 'Stromal fraction', 'Luminal fraction']

for data, name in zip(vals, names):
    plot_correlation(fIC_vals, data,  MR_map, name)